In [9]:
import pandas as pd
import numpy as np

# Load the input (recvd_serial_data) and output (data_source) CSV files
recvd_serial_data= pd.read_csv('6recvd_serial_data44.csv', header=None)
data_source = pd.read_csv('6data_source44.csv', header=None)



recvd_serial_data[0] = recvd_serial_data[0].str.strip()
recvd_serial_data[0] = recvd_serial_data[0].str.replace(' ', '')
recvd_serial_data[0] = recvd_serial_data[0].str.replace('i', 'j')

def to_complex(x):
    try:
        return complex(x)
    except ValueError:
        return np.nan


complex_numbers = recvd_serial_data[0].apply(to_complex)

# Convert recvd_serial_data into real and imaginary parts as separate features
X = pd.DataFrame({
    'real': complex_numbers.apply(np.real),
    'imag': complex_numbers.apply(np.imag)
})


# Convert the data_source into 4-bit binary outputs
y = pd.DataFrame(data_source[0].apply(lambda x: list(map(int, np.binary_repr(x, width=4)))).to_list())

# Save the processed input and output data as CSV
X.to_csv('input_data.csv', index=False)
y.to_csv('output_data.csv', index=False)

In [6]:
y.shape

(10000, 4)

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

X = pd.read_csv('input_data.csv')
y = pd.read_csv('output_data.csv')


# Normalize the input data (real and imaginary parts of QAM symbols)
scaler = MinMaxScaler(feature_range=(-1, 1) )
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets (80% for training)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the MLP model
model = MLPClassifier(hidden_layer_sizes=(64,16), max_iter=500, activation='relu', solver='adam', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Save the model
joblib.dump(model, 'qam_to_4bit_mlp_model3.pkl')



Test Accuracy: 99.35%


['qam_to_4bit_mlp_model3.pkl']

In [ ]:
test_input = pd.DataFrame([[2.5, -0.89]], columns=['real', 'imag']) # Create a DataFrame matching the format of X
print(model.predict(test_input))

[[1 0 1 1]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [11]:
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

model = joblib.load('qam_to_4bit_mlp_model3.pkl')

new_input_data = pd.read_csv('allBW_allSNR.csv', header=None)

new_input_data[0] = new_input_data[0].str.strip()
new_input_data[0] = new_input_data[0].str.replace(' ', '')
new_input_data[0] = new_input_data[0].str.replace('i', 'j')

def to_complex(x):
    try:
        return complex(x)
    except ValueError:
        return np.nan

complex_numbers = new_input_data[0].apply(to_complex)

scaler = MinMaxScaler(feature_range=(-1, 1))
predicted_bits_all = []

batch_size = 10000
num_batches = len(complex_numbers) // batch_size

for i in range(num_batches):
    batch = complex_numbers[i * batch_size: (i + 1) * batch_size]

    X = pd.DataFrame({
        'real': batch.apply(np.real),
        'imag': batch.apply(np.imag)
    })

    X_scaled = scaler.fit_transform(X)

    predicted_bits = model.predict(X_scaled)

    # Append the predicted bits to the result
    predicted_bits_all.append(predicted_bits)

# Combine all predicted bits into a single array (160000, 4)
predicted_bits_all = np.vstack(predicted_bits_all)

# Ensure the output shape is 160000 x 4
assert predicted_bits_all.shape == (780000, 4)

# Save the predicted bits to a CSV file
pd.DataFrame(predicted_bits_all).to_csv('6_44_all_b.csv', index=False)

